In [40]:
!chmod +x git.sh

In [14]:
import os

os.environ["HF_HOME"] = "/workspace/AAIPL/hf_cache"
os.environ["HF_DATASETS_CACHE"] = "/workspace/AAIPL/hf_cache/datasets"
os.environ["TRANSFORMERS_CACHE"] = "/workspace/AAIPL/hf_cache/models"

os.makedirs("/workspace/AAIPL/hf_cache/datasets", exist_ok=True)
os.makedirs("/workspace/AAIPL/hf_cache/models", exist_ok=True)


In [1]:
import json

FILE = "generated_mcq_dataset (1).json"

with open(FILE, "r") as f:
    data = json.load(f)

def validate_schema(item):
    if not isinstance(item, dict):
        return False
    
    required = {"topic", "question", "choices", "answer", "explanation"}
    if not required.issubset(item.keys()):
        return False
    
    if not isinstance(item["choices"], list) or len(item["choices"]) != 4:
        return False
    
    if item["answer"] not in ["A", "B", "C", "D"]:
        return False
    
    if not isinstance(item["question"], str) or len(item["question"]) < 20:
        return False
    
    return True

valid = [item for item in data if validate_schema(item)]

print("Total:", len(data))
print("Valid:", len(valid))
print("Invalid:", len(data) - len(valid))


Total: 1200
Valid: 1200
Invalid: 0


In [8]:
import json
import random
import re

INPUT_FILE = "generated_mcq_dataset (1).json"
OUTPUT_FILE = "a_agent_sft_balanced.jsonl"

with open(INPUT_FILE, "r") as f:
    dataset = json.load(f)

print("Loaded samples:", len(dataset))

balanced_data = []

for example in dataset:

    question = example["question"]
    choices = example["choices"]  # list of strings like "A) text"
    correct_letter = example["answer"]
    explanation = example["explanation"]

    # Extract letter + text
    parsed_choices = []
    for choice in choices:
        match = re.match(r"([ABCD])\)\s*(.*)", choice)
        if not match:
            continue
        parsed_choices.append(match.groups())

    if len(parsed_choices) != 4:
        continue

    option_dict = {letter: text for letter, text in parsed_choices}

    # Shuffle letters
    letters = ["A", "B", "C", "D"]
    new_letters = letters.copy()
    random.shuffle(new_letters)

    mapping = dict(zip(letters, new_letters))

    # Rebuild shuffled choices
    new_choices = []
    for old_letter in letters:
        new_letter = mapping[old_letter]
        new_choices.append(f"{new_letter}) {option_dict[old_letter]}")

    # Update correct answer
    new_correct_letter = mapping[correct_letter]

    # Build chat-format structure for SFT
    balanced_data.append({
        "messages": [
            {
                "role": "system",
                "content": "You are an expert logical reasoning assistant. Carefully analyze the MCQ and respond with the correct answer and a concise explanation in JSON format."
            },
            {
                "role": "user",
                "content": question + "\n\n" + "\n".join(new_choices)
            },
            {
                "role": "assistant",
                "content": json.dumps({
                    "answer": new_correct_letter,
                    "reasoning": explanation
                })
            }
        ]
    })

print("Balanced samples:", len(balanced_data))

# Save as JSONL
with open(OUTPUT_FILE, "w") as f:
    for item in balanced_data:
        f.write(json.dumps(item) + "\n")

print("Saved to:", OUTPUT_FILE)


Loaded samples: 1200
Balanced samples: 1200
Saved to: a_agent_sft_balanced.jsonl


In [9]:
from collections import Counter
import json

counts = Counter()

with open("a_agent_sft_balanced.jsonl") as f:
    for line in f:
        obj = json.loads(line)
        ans = json.loads(obj["messages"][2]["content"])["answer"]
        counts[ans] += 1

print(counts)


Counter({'C': 311, 'D': 304, 'B': 301, 'A': 284})


In [12]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer

# -------- LOCAL MODEL PATH --------

MODEL_ROOT = "/workspace/AAIPL/hf_models/huggingface"
MODEL_FOLDER = "models--Unsloth--Llama-3.1-8B-Instruct"
SNAPSHOTS = os.path.join(MODEL_ROOT, MODEL_FOLDER, "snapshots")

snapshot_hash = os.listdir(SNAPSHOTS)[0]
LOCAL_MODEL_PATH = os.path.join(SNAPSHOTS, snapshot_hash)

print("Loading model from:", LOCAL_MODEL_PATH)

DATA_PATH = "/workspace/AAIPL/a_agent_sft_balanced.jsonl"
dataset = load_dataset(
    "json",
    data_files=DATA_PATH,
    cache_dir="/workspace/AAIPL/hf_cache/datasets"
)["train"]

split_dataset = dataset.train_test_split(
    test_size=0.1,  # 10% validation
    seed=42
)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]


tokenizer = AutoTokenizer.from_pretrained(
    LOCAL_MODEL_PATH,
    local_files_only=True,
    use_fast=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    LOCAL_MODEL_PATH,
    torch_dtype=torch.bfloat16,
    local_files_only=True,
    device_map="auto"
)
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


Loading model from: /workspace/AAIPL/hf_models/huggingface/models--Unsloth--Llama-3.1-8B-Instruct/snapshots/4699cc75b550f9c6f3173fb80f4703b62d946aa5


Generating train split: 0 examples [00:00, ? examples/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
from peft import LoraConfig, get_peft_model


In [14]:
model = get_peft_model(model, peft_config)

print("LoRA model ready.")

LoRA model ready.


In [15]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="/workspace/AAIPL/a_agent_lora",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=5,
    save_strategy="epoch",
    do_eval=True,
    eval_strategy = "steps",
    eval_steps = 50,
    bf16=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="none",
    gradient_checkpointing=True,
)


trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset =val_dataset,
    args=training_args,
)


trainer.train()
trainer.save_model("/workspace/AAIPL/a_agent_lora_final")


Tokenizing train dataset:   0%|          | 0/1080 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1080 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,0.355300,0.348901,0.360281,137331.000000,0.874073
100,0.308500,0.315344,0.302131,273061.000000,0.883108
150,0.258400,0.306232,0.259197,408405.000000,0.887783
200,0.259400,0.302248,0.263603,545384.000000,0.889302


In [16]:
for log in trainer.state.log_history:
    if "eval_loss" in log:
        print(log)


{'eval_loss': 0.3489009737968445, 'eval_runtime': 1.1542, 'eval_samples_per_second': 103.971, 'eval_steps_per_second': 12.996, 'eval_entropy': 0.3602811018625895, 'eval_num_tokens': 137331.0, 'eval_mean_token_accuracy': 0.87407306432724, 'epoch': 0.7407407407407407, 'step': 50}
{'eval_loss': 0.3153443932533264, 'eval_runtime': 1.1485, 'eval_samples_per_second': 104.489, 'eval_steps_per_second': 13.061, 'eval_entropy': 0.30213125546773273, 'eval_num_tokens': 273061.0, 'eval_mean_token_accuracy': 0.8831080238024394, 'epoch': 1.474074074074074, 'step': 100}
{'eval_loss': 0.3062315285205841, 'eval_runtime': 1.1506, 'eval_samples_per_second': 104.295, 'eval_steps_per_second': 13.037, 'eval_entropy': 0.25919678807258606, 'eval_num_tokens': 408405.0, 'eval_mean_token_accuracy': 0.887783149878184, 'epoch': 2.2074074074074073, 'step': 150}
{'eval_loss': 0.3022478222846985, 'eval_runtime': 1.1489, 'eval_samples_per_second': 104.446, 'eval_steps_per_second': 13.056, 'eval_entropy': 0.263602882623

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/workspace/AAIPL/a_agent_lora_final"

model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
  

In [19]:
model.to("cuda")
print(model.device)


cuda:0


In [20]:
import torch
import json
from tqdm import tqdm

model.eval()

batch_size = 8
correct = 0
total = 0

for i in tqdm(range(0, len(val_dataset), batch_size)):

    batch = val_dataset.select(
        range(i, min(i+batch_size, len(val_dataset)))
    )

    prompts = []
    true_answers = []

    for example in batch:

        messages = example["messages"]
        system_msg = messages[0]
        user_msg = messages[1]
        assistant_msg = messages[2]

        true_answer = json.loads(
            assistant_msg["content"]
        )["answer"]

        true_answers.append(true_answer)

        prompt = tokenizer.apply_chat_template(
            [system_msg, user_msg],
            tokenize=False,
            add_generation_prompt=True
        )

        prompts.append(prompt)

    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=60,        # enough for JSON
            do_sample=False,
            temperature=0.0
        )

    decoded = tokenizer.batch_decode(
        outputs,
        skip_special_tokens=True
    )

    for text, true_ans in zip(decoded, true_answers):

        # Extract only generated part
        # (everything after prompt)
        try:
            json_start = text.find("{")
            json_str = text[json_start:]
            pred_json = json.loads(json_str)

            predicted_answer = pred_json["answer"]

            if predicted_answer == true_ans:
                correct += 1
        except:
            pass

        total += 1

accuracy = correct / total
print("Validation Accuracy:", accuracy)


100%|██████████| 15/15 [00:22<00:00,  1.53s/it]

Validation Accuracy: 0.6083333333333333


In [28]:
import json
import os
from collections import defaultdict

INPUT_FILE = "generated_mcq_dataset (1).json"
OUTPUT_FILE = "outputs/filtered_questions.json"

# Load original dataset
with open(INPUT_FILE, "r") as f:
    dataset = json.load(f)

# Group by topic
topic_map = {}

for item in dataset:
    topic = item["topic"]
    if topic not in topic_map:
        topic_map[topic] = item  # take first occurrence

print("Total topics found:", len(topic_map))

# Convert to answer_agent expected format
filtered_questions = []

for topic, item in topic_map.items():
    filtered_questions.append({
        "topic": topic,
        "question": item["question"],
        "choices": item["choices"]  # already formatted as A) ...
    })

# Ensure outputs folder exists
os.makedirs("outputs", exist_ok=True)

# Save
with open(OUTPUT_FILE, "w") as f:
    json.dump(filtered_questions, f, indent=4)

print(f"Created {len(filtered_questions)} questions in {OUTPUT_FILE}")


Total topics found: 4
Created 4 questions in outputs/filtered_questions.json


In [29]:
!python -m agents.answer_agent \
  --input_file outputs/filtered_questions.json \
  --output_file outputs/answers.json \
  --verbose


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
STEPS:   0%|                                           | 0/1 [00:00<?, ?batch/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
STEPS: : 2batch [00:18,  9.08s/batch]                                           

=== Question 1 ===
Question: If all cats are animals that can climb trees, and some animals that can climb trees are not dogs, and no dogs are cats, which of the following must be true?
Expected: N/A
Model Answer:
{"answer": "C", "reasoning": "Since all cats can climb trees but no dogs are cats, it follows logically that no animals that can climb trees could also be dogs."}

=== Question 2 ===
Question: In a circular arrangement of

## Q-Agent

In [2]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import PPOTrainer, PPOConfig

MODEL_ROOT = "/workspace/AAIPL/hf_models/huggingface"
MODEL_FOLDER = "models--Unsloth--Llama-3.1-8B-Instruct"
SNAPSHOTS = os.path.join(MODEL_ROOT, MODEL_FOLDER, "snapshots")

snapshot_hash = os.listdir(SNAPSHOTS)[0]
LOCAL_MODEL_PATH = os.path.join(SNAPSHOTS, snapshot_hash)

print("Loading model from:", LOCAL_MODEL_PATH)

tokenizer = AutoTokenizer.from_pretrained(
    LOCAL_MODEL_PATH,
    local_files_only=True,
    use_fast=True
)
model = AutoModelForCausalLM.from_pretrained(
    LOCAL_MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto"
)

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj","k_proj","o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading model from: /workspace/AAIPL/hf_models/huggingface/models--Unsloth--Llama-3.1-8B-Instruct/snapshots/4699cc75b550f9c6f3173fb80f4703b62d946aa5


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


In [3]:
def build_prompt(topic: str) -> str:

    import random

    SYLLOGISM_DOMAINS = [
        "Professions",
        "Scientific fields",
        "Companies and organizations",
        "Geographic locations",
        "Vehicles and machines",
        "Academic disciplines",
        "Musical genres",
        "Technologies",
        "Historical groups",
        "Kitchen utensils",
        "Sports teams"
    ]

    topic_constraints = ""

    if topic == "Syllogisms":
        domain = random.choice(SYLLOGISM_DOMAINS)
    
        topic_constraints = f"""
    STRICT LOGIC FORMAT (MANDATORY):
    
    Each premise must follow EXACTLY one of these forms:
    
    - All A are B.
    - No A are B.
    - Some A are B.
    - Some A are not B.
    
    Replace A and B with categorical nouns from this domain: {domain}.
    Use short abstract category names (e.g., "Architects", "Engineers", "Universities").
    
    Do NOT use:
    - Explanatory phrases
    - Relative clauses (that, which, who)
    - Property-based statements (e.g., "companies that own banks")
    - Real-world factual claims
    
    Write EXACTLY FOUR premises labeled:
    
    Premise 1:
    Premise 2:
    Premise 3:
    Premise 4:
    
    The final question must be:
    "Which of the following conclusions logically follows?"
    
    All answer choices must also strictly follow the same categorical structure.
    
    The correct conclusion must be logically deducible by formal syllogistic reasoning.
    Distractors must include:
    - One with reversed quantifier
    - One with undistributed middle error
    - One invalid existential leap
    
    Do NOT invent factual reasoning.
    Only use formal logical deduction.
    """


    elif topic == "Seating Arrangements (Circular and Linear)":

        topic_constraints = """
- Use EXACTLY 7 individuals with distinct names.
- Clearly state whether arrangement is Circular or Linear at the beginning.
- Include at least:
    • Two adjacency constraints,
    • One directional left/right constraint,
    • One negative constraint (not adjacent / not opposite),
    • One positional constraint relative to two people.
- At least THREE constraints must interact to determine the answer.
- Ensure the solution is logically unique.
- Avoid simple linear chaining placements.
"""

    elif topic == "Blood Relations and Family Tree":

        topic_constraints = """
- Include EXACTLY 6 individuals across at least THREE generations.
- Include at least one marriage relation.
- Use indirect relations such as maternal uncle, paternal aunt, niece, grandson.
- The final question must require tracing through at least TWO inferential steps.
- Avoid trivial parent-child-only deductions.
- The answer must NOT be derivable from a single sentence alone.
"""

    elif topic == "Mixed Series (Alphanumeric)":

        topic_constraints = """
- Sequence must contain at least 5 visible terms with one missing term.
- Must combine BOTH:
    • A non-linear numerical rule (multiplication, division, squares, alternating operations, or layered pattern).
    • A non-linear letter progression (variable jumps, cyclic shifts, alternating increments).
- DO NOT use simple +1 or constant-addition patterns.
- The pattern must require identifying at least TWO interacting rules.
- Letters and numbers must NOT change independently in a trivial manner.
- The answer must require analyzing BOTH components together.
"""

    prompt = f"""
You are generating ONE difficult logical reasoning MCQ.

Topic: {topic}

Requirements:
{topic_constraints}

General Rules:
- Exactly 4 answer choices labeled A, B, C, D.
- Exactly ONE correct answer.
- No duplicate choices.
- The distractors must be logically plausible.
- Difficulty must be higher than typical competitive exam questions.
- Explanation must be concise (maximum 80 words).
- Output ONLY valid JSON.
- Do NOT include markdown.
- Do NOT include text outside JSON.
- Generate completely NEW content.
- Internally verify the answer before outputting.

Return JSON in this exact structure:

{{
    "topic": "{topic}",
    "question": "Your question ending with a question mark?",
    "choices": [
        "A) First option",
        "B) Second option",
        "C) Third option",
        "D) Fourth option"
    ],
    "answer": "A",
    "explanation": "Concise reasoning explanation."
}}
"""
    return prompt


In [4]:
def generate_question_clean(topic):
    prompt = build_prompt(topic)

    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(chat, return_tensors="pt").to(model.device)

    output_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty = 1.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    new_tokens = output_ids[0][inputs["input_ids"].shape[-1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    # Extract only JSON block
    import re, json
    match = re.search(r'\{.*\}', response, re.DOTALL)
    if match:
        return match.group(0)

    return response


In [48]:
q = generate_question_clean("Syllogisms")
q


'{\n    "topic": "Syllogisms",\n    "question": "All Geologists are Researchers. Some Researchers are Professors. Some Professors are Scientists. Which of the following conclusions logically follows?",\n    "choices": [\n        "A) Some Scientists are Geologists.",\n        "B) All Scientists are Researches.",\n        "C) Some Researchers are Not Scientists.",\n        "D) All Geologists are Scientists."\n    ],\n    "answer": "A",\n    "explanation": "From \'Some Researchers are Professors\' we get \'Some Professors are Researchers\'. Then using transitivity with \'All Geologists are Researchers\', it\'s inferred as some geologist is researcher; then applying\'some researchers are professors\', therefore some geologists are professors; since all scientists are not explicitly stated to have any subset or superset relationship so its safe to conclude\'some scientists are professors\'; Now given this result along with last statement some professoer are scientist, hence inferencing\'som

In [5]:
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load dataset
dataset = load_dataset(
    "json",
    data_files="a_agent_sft_balanced.jsonl"
)["train"]

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./q_agent_sft",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    report_to="none"
)

# IMPORTANT: use processing_class instead of tokenizer
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset,
    args=training_args,
)

trainer.train()
trainer.save_model("./q_agent_sft_final")


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


KeyboardInterrupt: 

In [15]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch

MODEL_ROOT = "/workspace/AAIPL/hf_models/huggingface"
MODEL_FOLDER = "models--Unsloth--Llama-3.1-8B-Instruct"
SNAPSHOTS = os.path.join(MODEL_ROOT, MODEL_FOLDER, "snapshots")

snapshot_hash = os.listdir(SNAPSHOTS)[0]
LOCAL_MODEL_PATH = os.path.join(SNAPSHOTS, snapshot_hash)

print("Loading model from:", LOCAL_MODEL_PATH)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    LOCAL_MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(
    base_model,
    "./q_agent_sft_final",
    is_trainable=True  # 🔥 THIS IS CRUCIAL
)

model.train()
model.print_trainable_parameters()

Loading model from: /workspace/AAIPL/hf_models/huggingface/models--Unsloth--Llama-3.1-8B-Instruct/snapshots/4699cc75b550f9c6f3173fb80f4703b62d946aa5


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


In [16]:
from agents.answer_model_1 import AAgent

a_model = AAgent()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
import json

def run_a_agent(question_json):
    question = question_json["question"]
    choices = question_json["choices"]

    formatted_choices = " ".join(choices)

    prompt = f"""
Question: {question}
Choices: {formatted_choices}

Respond ONLY with JSON:
{{"answer": "A"}}
"""

    response, _, _ = a_model.generate_response(
        prompt,
        system_prompt="You are a precise logical reasoning solver.",
        max_new_tokens=512,
        temperature=0.0,
        do_sample=False
    )

    try:
        parsed = json.loads(response)
        return parsed.get("answer", None)
    except:
        return None


In [18]:
sample = generate_question_clean("Syllogisms")
question_json = json.loads(sample)
print(run_a_agent(question_json))


A


In [19]:
def generate_sample(topic):
    prompt = build_prompt(topic)

    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(chat, return_tensors="pt").to(model.device)

    output_ids = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    full_sequence = output_ids[0]

    return inputs["input_ids"][0], full_sequence


In [20]:
def compute_logprob(prompt_ids, full_sequence_ids):
    with torch.enable_grad():

        outputs = model(full_sequence_ids.unsqueeze(0))

        logits = outputs.logits

        # Shift for next-token prediction
        shift_logits = logits[:, :-1, :]
        shift_labels = full_sequence_ids[1:].unsqueeze(0)

        log_probs = torch.log_softmax(shift_logits, dim=-1)

        selected_log_probs = log_probs.gather(
            2,
            shift_labels.unsqueeze(-1)
        ).squeeze(-1)

        total_log_prob = selected_log_probs.sum()

        return total_log_prob


In [21]:
def compute_reward(q_json):
    if not isinstance(q_json, dict):
        return -3

    if len(q_json.get("choices", [])) != 4:
        return -3

    if "answer" not in q_json:
        return -3

    # Ask A-agent
    a_pred = run_a_agent(q_json)

    if a_pred is None:
        return -2

    if a_pred == q_json["answer"]:
        return 0.5      # easy question
    else:
        return 2.0      # fooled A-agent


In [22]:
import random

topics = [
    "Syllogisms",
    "Seating Arrangements (Circular and Linear)",
    "Blood Relations and Family Tree",
    "Mixed Series (Alphanumeric)"
]

optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=5e-6
)

model.train()

for step in range(30):

    topic = random.choice(topics)

    prompt_ids, full_sequence_ids = generate_sample(topic)

    decoded = tokenizer.decode(
        full_sequence_ids[prompt_ids.shape[-1]:],
        skip_special_tokens=True
    )

    try:
        q_json = json.loads(decoded)
        reward = compute_reward(q_json)
    except:
        reward = -3

    log_prob = compute_logprob(prompt_ids, full_sequence_ids)

    loss = -reward * log_prob

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"Step {step} | Reward: {reward}")


Step 0 | Reward: 0.5
Step 1 | Reward: 0.5
Step 2 | Reward: 2.0
Step 3 | Reward: 0.5
Step 4 | Reward: 2.0
Step 5 | Reward: 2.0
Step 6 | Reward: 2.0
Step 7 | Reward: 0.5
Step 8 | Reward: 2.0
Step 9 | Reward: 0.5
Step 10 | Reward: 2.0
Step 11 | Reward: 2.0
Step 12 | Reward: 2.0
Step 13 | Reward: 0.5
Step 14 | Reward: 0.5
Step 15 | Reward: 0.5
Step 16 | Reward: 0.5
Step 17 | Reward: 2.0
Step 18 | Reward: 0.5
Step 19 | Reward: 0.5
Step 20 | Reward: 2.0
Step 21 | Reward: 0.5
Step 22 | Reward: 0.5
Step 23 | Reward: 2.0
Step 24 | Reward: 2.0
Step 25 | Reward: 0.5
Step 26 | Reward: 0.5
Step 27 | Reward: 2.0
Step 28 | Reward: 2.0
Step 29 | Reward: 2.0


In [23]:
model.save_pretrained("./q_agent_rl_final")


In [24]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_

In [25]:
def generate_question_eval(topic):
    prompt = build_prompt(topic)

    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(chat, return_tensors="pt").to(model.device)

    output_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated = output_ids[0][inputs["input_ids"].shape[-1]:]

    decoded = tokenizer.decode(
        generated,
        skip_special_tokens=True
    )

    return decoded


In [26]:
import json
import random

fail_count = 0
total = 10

topics = [
    "Syllogisms",
    "Seating Arrangements (Circular and Linear)",
    "Blood Relations and Family Tree",
    "Mixed Series (Alphanumeric)"
]

for i in range(total):

    topic = random.choice(topics)
    decoded = generate_question_eval(topic)

    try:
        q_json = json.loads(decoded)
    except:
        print("Invalid JSON")
        continue

    a_pred = run_a_agent(q_json)

    correct = q_json["answer"]

    print(f"\nQuestion {i+1}")
    print("Topic:", topic)
    print("Correct:", correct)
    print("A-Agent Pred:", a_pred)

    if a_pred != correct:
        fail_count += 1

print("\nA-Agent Accuracy:", (total - fail_count) / total)
print("A-Agent Failure Rate:", fail_count / total)



Question 1
Topic: Syllogisms
Correct: A
A-Agent Pred: A

Question 2
Topic: Seating Arrangements (Circular and Linear)
Correct: A
A-Agent Pred: A

Question 3
Topic: Blood Relations and Family Tree
Correct: C) Grandson
A-Agent Pred: A

Question 4
Topic: Syllogisms
Correct: A
A-Agent Pred: A

Question 5
Topic: Syllogisms
Correct: B
A-Agent Pred: A

Question 6
Topic: Blood Relations and Family Tree
Correct: B
A-Agent Pred: A

Question 7
Topic: Mixed Series (Alphanumeric)
Correct: C
A-Agent Pred: A

Question 8
Topic: Blood Relations and Family Tree
Correct: A
A-Agent Pred: B

Question 9
Topic: Seating Arrangements (Circular and Linear)
Correct: A
A-Agent Pred: A

Question 10
Topic: Blood Relations and Family Tree
Correct: A
A-Agent Pred: A

A-Agent Accuracy: 0.5
A-Agent Failure Rate: 0.5


In [29]:
!python -m agents.question_agent \
    --num_questions 20 \
    --output_file outputs/questions.json \
    --batch_size 5 \
    --verbose


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading base model from: /workspace/AAIPL/hf_models/huggingface/models--Unsloth--Llama-3.1-8B-Instruct/snapshots/4699cc75b550f9c6f3173fb80f4703b62d946aa5
`torch_dtype` is deprecated! Use `dtype` instead!
STEPS: 100%|██████████████████████████████████████| 4/4 [00:45<00:00, 11.43s/it]
Generated 20 questions!
{
  "topic": "Mixed Series (Alphanumeric)",
  "question": "T5O, U6P, V7Q, W8R, _____",
  "choices": ["A) X9S", "B) Y10T", "C) W9R", "D) X10S"],
  "answer": "D",
  "explanation": "The series involves two interleaved sequences: the first and third letters are alphabetical in order, and the second and fourth letters are in a numerical order. The first letter increases by one in the alphabet, and the second letter increases by one in the number sequence. Hence X10S."
}
{
  "topic"

In [37]:
from typing import List, Dict, Any

MODEL_ROOT = "/workspace/AAIPL/hf_models/huggingface"

MODEL_FOLDER = "models--Qwen--Qwen2.5-14B-Instruct"
SNAPSHOTS = os.path.join(MODEL_ROOT, MODEL_FOLDER, "snapshots")

# Get actual snapshot folder
snapshot_hash = os.listdir(SNAPSHOTS)[0]
LOCAL_MODEL_PATH = os.path.join(SNAPSHOTS, snapshot_hash)

print("Loading model from:", LOCAL_MODEL_PATH)

tokenizer = AutoTokenizer.from_pretrained(
    LOCAL_MODEL_PATH,
    local_files_only=True
)
def count_tokens_q(text: str) -> int:
    return len(tokenizer.encode(text, add_special_tokens=False))

def filter_questions(questions: List[str|Dict[str, str|Any]]) -> List[Dict[str, str|Any]]:
    def basic_checks(q2: Dict[str, str])->bool:
        # check required keys
        required_keys = ['topic', 'question', 'choices', 'answer']
        if all((key in q2) for key in required_keys):
            # check choices format
            checks = all(isinstance(choice, str) and len(choice) > 2 and choice[0].upper() in 'ABCD' for choice in q2['choices'])
            if isinstance(q2['choices'], list) and len(q2['choices']) == 4 and checks:
                # check answer format
                # Check token length
                check_len = sum(count_tokens_q(q2[k]) for k in ['question', 'answer'])
                check_len += sum(count_tokens_q(choice) for choice in q2['choices']) - 15
                if check_len < 130:
                    if check_len + count_tokens_q(q2.get('explanation', 'None')) <= 1024:
                        # Extra Checks: (PLUS checks) len(q2['answer']) == 1 and q2['answer'].upper() in 'ABCD':
                        if isinstance(q2['answer'], str):
                            return True
        return False
    correct_format_question = []
    for i, q in enumerate(questions):
        if isinstance(q, dict):
            if basic_checks(q):
                correct_format_question.append(q)
        elif isinstance(q, str):
            try:
                q1 = json.loads(q)
                if basic_checks(q1):
                    correct_format_question.append(q1)
            except json.JSONDecodeError:
                # If JSON decoding fails, skip this answer
                print(f"Skipping invalid JSON at index {i}: {q}")
                continue
        else:
            continue
    if len(correct_format_question) >= 0.5 * len(questions):
        return correct_format_question
    return list()

Loading model from: /workspace/AAIPL/hf_models/huggingface/models--Qwen--Qwen2.5-14B-Instruct/snapshots/cf98f3b3bbb457ad9e2bb7baf9a0125b6b88caa8


In [38]:
with open("outputs/questions.json", "r") as f:
    questions = json.load(f)

filtered_questions = filter_questions(questions)

with open("outputs/filtered_questions.json", "w") as f:
    json.dump(filtered_questions, f, indent=4)

print(f"Number of questions: {len(questions)}")
print(f"Number of filtered questions: {len(filtered_questions)}")

Number of questions: 20
Number of filtered questions: 11


In [39]:
!python -m agents.question_agent \
    --num_questions 20 \
    --output_file outputs/filtered_questions.json \
    --batch_size 5 \
    --verbose


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading base model from: /workspace/AAIPL/hf_models/huggingface/models--Unsloth--Llama-3.1-8B-Instruct/snapshots/4699cc75b550f9c6f3173fb80f4703b62d946aa5
`torch_dtype` is deprecated! Use `dtype` instead!
STEPS: 100%|██████████████████████████████████████| 4/4 [00:42<00:00, 10.61s/it]
Generated 20 questions!
{
  "topic": "Blood Relations and Family Tree/Family tree logic",
  "question": "Rahul says, “My mother’s husband is the son of my father’s mother’s husband.” How is Rahul’s mother related to the man in the picture?",
  "choices": ["A) A) Wife", "B) B) Daughter", "C) C) Granddaughter", "D) D) Sister-in-law"],
  "answer": "B",
  "explanation": "Rahul's'mother's husband' is his father. His 'father's mother' is his paternal grandmother. The 'husband' of his paternal grandmother i